## 4장 다변량 선형회귀

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.0.0-alpha0


* 이전까지는 선형회귀(linear regression)이라고 하면 y = ax + b 의 개념으로 봤다.
>
$$ H(x) = w x + b $$

* 이제는 '다변량' 이라는 개념으로 식을 봐야 한다. y = ax1 + bx2 + cx3 + d 이런 느낌
$$ H(x_1, x_2, x_3) = w_1 x_1 + w_2 x_2 + w_3 x_3 + b $$

* 변수가 많아지면서 기존의 방식으로는 계산을 진행하기 어렵다.
* 그래서 Matrix의 개념을 도입하는 거다.

$$ w_1 x_1 + w_2 x_2 + w_3 x_3 $$
>
$$ = \begin{pmatrix} w_{ 1 } & w_{ 2 } & w_{ 3 } \end{pmatrix}\cdot \begin{pmatrix} x_{ 1 } \\ x_{ 2 } \\ x_{ 3 } \end{pmatrix} $$
>
$$ = WX $$

#### 그런데 W * X와 X * W 중 뭐가 더 나을까?

* instance는 여러 값이 나오지만, weight는 고정적이다.
* 그래서 X * W로 보는 게 깔끔(?!)
* Tensorflow는 H(X) = XW로 계산

### 예시 도전!

* 함수식과 Matrix를 직접 비교해보자

x1 | x2 | y
---- | ---- | ----
1  |  0  |  1
0  |  2  |  2
3  |  0  |  3
0  |  4  |  4
5  |  0  |  5

### 함수식

In [5]:
tf.random.set_seed(0)  # 계산 값 동일하도록 seed값 설정

In [6]:
x1 = [1, 0, 3, 0, 5]
x2 = [0, 2, 0, 4, 0]
y  = [1, 2, 3, 4, 5]

In [8]:
W1 = tf.Variable(tf.random.uniform((1,), -10.0, 10.0)) # uniform(균등)한 형태로 -10부터 10사이에서 랜덤한 변수 설정
W2 = tf.Variable(tf.random.uniform((1,), -10.0, 10.0))
b  = tf.Variable(tf.random.uniform((1,), -10.0, 10.0))

In [9]:
learning_rate = tf.Variable(0.001) # learning rate는 0.001로 할까

In [15]:
for i in range(1000+1): # 1001번의 for문을 돌린다.
    with tf.GradientTape() as tape: # with 문 안의 함수를 tape에 기록함
        hypothesis = W1 * x1 + W2 * x2 + b
        cost = tf.reduce_mean(tf.square(hypothesis - y))
    W1_grad, W2_grad, b_grad = tape.gradient(cost, [W1, W2, b]) # cost함수 내의 W1, W2, b 의 미분값을 왼쪽으로 삽입
    W1.assign_sub(learning_rate * W1_grad) # W1에 learning_rate * 지금 W1의 기울기를 넣기
    W2.assign_sub(learning_rate * W2_grad) # W2에 learning_rate * 지금 W2의 기울기를 넣기
    b.assign_sub(learning_rate * b_grad)   # b에  learning_eate * b의 기울기를 넣기
    
    # 한 번 결과를 볼까
    
    if i % 50 == 0: # 50번에 한 번씩 아래로 ㄱㄱ
        print("{:5} | {:10.6f} | {:10.4f} | {:10.4f} | {:10.6f}".format(
          i, cost.numpy(), W1.numpy()[0], W2.numpy()[0], b.numpy()[0]))

    0 | 1256.619385 |    -9.1905 |    -1.9179 |  -8.748963
   50 | 287.577789 |    -3.0720 |    -0.2124 |  -6.530516
  100 |  69.545204 |    -0.2508 |     0.7721 |  -5.404896
  150 |  19.368086 |     1.0387 |     1.3499 |  -4.809908
  200 |   7.329938 |     1.6190 |     1.6926 |  -4.475818
  250 |   4.214845 |     1.8726 |     1.8963 |  -4.272120
  300 |   3.292675 |     1.9766 |     2.0163 |  -4.134976
  350 |   2.951777 |     2.0129 |     2.0849 |  -4.032729
  400 |   2.781655 |     2.0190 |     2.1218 |  -3.949406
  450 |   2.668293 |     2.0120 |     2.1388 |  -3.876784
  500 |   2.576570 |     1.9996 |     2.1433 |  -3.810526
  550 |   2.494502 |     1.9851 |     2.1400 |  -3.748281
  600 |   2.417632 |     1.9701 |     2.1318 |  -3.688743
  650 |   2.344183 |     1.9551 |     2.1204 |  -3.631163
  700 |   2.273401 |     1.9403 |     2.1072 |  -3.575097
  750 |   2.204935 |     1.9257 |     2.0929 |  -3.520278
  800 |   2.138608 |     1.9115 |     2.0779 |  -3.466537
  850 |   2.0

### 행렬식(Matrix)

In [16]:
tf.random.set_seed(0)

In [18]:
x = [
    [1., 0., 3., 0., 5.],  # 행렬 형태로 데이터를 저장
    [0., 2., 0., 4., 0.]
]
y = [1, 2, 3, 4, 5]

In [19]:
W = tf.Variable(tf.random.uniform((1, 2), -1.0, 1.0)) # (1, 2) # 1행 2열
b  = tf.Variable(tf.random.uniform((1, ), -1.0, 1.0))

In [20]:
learning_rate = tf.Variable(0.001)

In [22]:
for i in range(1000+1):
    with tf.GradientTape() as tape:
        hypothesis = tf.matmul(W, x) + b # 3장에서도 언급했듯, matmul은 행렬곱이다.
        cost = tf.reduce_mean(tf.square(hypothesis - y))
        
        W_grad, b_grad = tape.gradient(cost, [W, b])
        W.assign_sub(learning_rate * W_grad)
        b.assign_sub(learning_rate * b_grad)
        
    if i % 50 == 0:
        print("{:5} | {:10.6f} | {:10.4f} | {:10.4f} | {:10.6f}".format(
          i, cost.numpy(), W.numpy()[0][0], W.numpy()[0][1],b.numpy()[0]))        

    0 |   0.067913 |     0.8377 |     0.8072 |   0.617546
   50 |   0.065871 |     0.8401 |     0.8102 |   0.608213
  100 |   0.063892 |     0.8425 |     0.8131 |   0.599015
  150 |   0.061971 |     0.8449 |     0.8159 |   0.589952
  200 |   0.060109 |     0.8473 |     0.8187 |   0.581024
  250 |   0.058302 |     0.8496 |     0.8215 |   0.572229
  300 |   0.056550 |     0.8519 |     0.8242 |   0.563567
  350 |   0.054850 |     0.8541 |     0.8269 |   0.555034
  400 |   0.053202 |     0.8563 |     0.8295 |   0.546631
  450 |   0.051603 |     0.8585 |     0.8321 |   0.538354
  500 |   0.050052 |     0.8606 |     0.8346 |   0.530203
  550 |   0.048548 |     0.8627 |     0.8371 |   0.522174
  600 |   0.047088 |     0.8648 |     0.8396 |   0.514268
  650 |   0.045673 |     0.8669 |     0.8420 |   0.506481
  700 |   0.044300 |     0.8689 |     0.8444 |   0.498812
  750 |   0.042969 |     0.8709 |     0.8468 |   0.491259
  800 |   0.041678 |     0.8728 |     0.8491 |   0.483820
  850 |   0.04

#### Hypothesis 에 bias를 넣는다면

In [2]:
import tensorflow as tf

In [3]:
X = [
    [1., 1., 1., 1., 1.],
    [1., 0., 3., 0., 5.],
    [0., 2., 0., 4., 0.]
]
y = [1, 2, 3, 4, 5]

In [7]:
W = tf.Variable(tf.random.uniform((1, 3), -1.0, 1.0)) # [1, 3]의 틀을 만들고 bias를 없앰

learning_rate = 0.001
optimizer = tf.keras.optimizers.SGD(learning_rate) # 갑분 케라스...?
    # SGD는 확률적 경사하강법으로 keras로 정말 간단하게 쓸 수 있음

for i in range(1000+1):
    with tf.GradientTape() as tape:
        hypothesis = tf.matmul(W, X) # W와 X를 Matrix 곱 / 여기서는 b를 신경 안 씀
        cost = tf.reduce_mean(tf.square(hypothesis - y))
    
    # W의 기울기를 구해보면?
    grads = tape.gradient(cost, [W])
    optimizer.apply_gradients(grads_and_vars = zip(grads,[W]))
    
    if i % 50 == 0: # 50번씩 마다 아래로 내려감
        print("{:5} | {:10.6f} | {: 10.4f} | {: 10.4f} | {:10.4f}".format(
        i, cost.numpy(), W.numpy()[0][0], W.numpy()[0][1], W.numpy()[0][2]))
        

    0 |  24.178761 |     0.7083 |    -0.9680 |     0.2746
   50 |   5.537626 |     0.9666 |    -0.0852 |     0.4291
  100 |   1.427081 |     1.0807 |     0.3289 |     0.5153
  150 |   0.512747 |     1.1263 |     0.5241 |     0.5657
  200 |   0.303499 |     1.1395 |     0.6171 |     0.5967
  250 |   0.250722 |     1.1371 |     0.6626 |     0.6171
  300 |   0.233131 |     1.1273 |     0.6860 |     0.6314
  350 |   0.223717 |     1.1141 |     0.6991 |     0.6421
  400 |   0.216380 |     1.0992 |     0.7075 |     0.6507
  450 |   0.209707 |     1.0836 |     0.7136 |     0.6580
  500 |   0.203352 |     1.0678 |     0.7186 |     0.6644
  550 |   0.197222 |     1.0519 |     0.7232 |     0.6703
  600 |   0.191288 |     1.0362 |     0.7275 |     0.6758
  650 |   0.185536 |     1.0206 |     0.7317 |     0.6810
  700 |   0.179959 |     1.0052 |     0.7358 |     0.6860
  750 |   0.174550 |     0.9901 |     0.7397 |     0.6909
  800 |   0.169304 |     0.9751 |     0.7437 |     0.6957
  850 |   0.16

### Prediction exam score (시험 성적 맞춰보기)

x1 (quiz 1) | x2 (quiz 2) | x3 (mid 1) | Y (final)
---- | ---- | ----| ----
73 | 80 | 75 | 152
93 | 88 | 93 | 185
89 | 91 | 90 | 180
96 | 98 | 100 | 196
73 | 66 | 70 | 142

In [9]:
tf.random.set_seed(0) # 값 고정

#### X값, 그리고 라벨링

In [10]:
x1 = [73., 93., 89., 96., 73.]
x2 = [80., 88., 91., 98., 66.]
x3 = [75., 93., 90., 100., 70.]
y = [152., 185., 180., 196., 142.]

#### weight 초기값 설정

In [11]:
w1 = tf.Variable(10.)
w2 = tf.Variable(10.)
w3 = tf.Variable(10.)
b  = tf.Variable(10.)

#### 식 구성

In [15]:
learning_rate = 0.00001

for i in range(1000+1):
    #GradientDescent를 구하기 위해서 GradientTape함수를 쓸 거다
    with tf.GradientTape() as tape:
        hypothesis = w1 * x1 + w2 * x2 + w3 * x3 + b
        cost = tf.reduce_mean(tf.square(hypothesis - y))
    
    # cost 함수 안의 w1, w2, w3, b 값들의 미분값을 왼쪽에 삽입
    W1_grad, W2_grad, W3_grad, b_grad = tape.gradient(cost, [w1, w2, w3, b])
    
    #for문마다 각 변수값을 업데이트 해준다.
    
    w1.assign_sub(learning_rate * W1_grad)
    w2.assign_sub(learning_rate * W2_grad)
    w3.assign_sub(learning_rate * W3_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 50 == 0:
        print("{:5} | {:12.4f}".format(i, cost.numpy()))

    0 | 1816078.3750
   50 |       1.9075
  100 |       1.8760
  150 |       1.8453
  200 |       1.8155
  250 |       1.7864
  300 |       1.7580
  350 |       1.7304
  400 |       1.7034
  450 |       1.6772
  500 |       1.6516
  550 |       1.6266
  600 |       1.6023
  650 |       1.5787
  700 |       1.5556
  750 |       1.5331
  800 |       1.5111
  850 |       1.4898
  900 |       1.4689
  950 |       1.4486
 1000 |       1.4288


### 다변량 회귀 분석 (방법 1, 무식 버전)

In [2]:
# 데이터 집어넣기
x1 = [73., 93., 89., 96., 73.]
x2 = [80., 88., 91., 98., 66.]
x3 = [75., 93., 90., 100., 70.]
Y = [152., 185., 180., 196., 142.]

In [4]:
# weight값 랜덤하게 넣기
w1 = tf.Variable(tf.random.normal((1,)))
w2 = tf.Variable(tf.random.normal((1,)))
w3 = tf.Variable(tf.random.normal((1,)))
b  = tf.Variable(tf.random.normal((1,)))

In [5]:
# learning_rate 설정
learning_rate = 0.00001

In [8]:
# 식 구성
for i in range(1000+1):
    with tf.GradientTape() as tape:
        hypothesis = w1 * x1 + w2 *x2 + w3 * x3 + b
        cost = tf.reduce_mean(tf.square(hypothesis - Y))
        # hypothesis와 cost 함수를 tape 안에 집어넣는다.
    w1_grad, w2_grad, w3_grad, b_grad = tape.gradient(cost, [w1, w2, w3, b])
    
    # w1, w2, w3, b의 기울기를 통해 각 값들을 변경하자
    
    w1.assign_sub(learning_rate * w1_grad)
    w2.assign_sub(learning_rate * w2_grad)
    w3.assign_sub(learning_rate * w3_grad)
    b.assign_sub(learning_rate * b_grad)
    
    # 그리고 for문이 50번씩 돌때마다 아래로 내려가서 print
    if i % 50 == 0:
        print("{:5} | {:12.4f}".format(i, cost.numpy()))

    0 |   54173.1680
   50 |      13.0500
  100 |      12.7070
  150 |      12.3732
  200 |      12.0483
  250 |      11.7321
  300 |      11.4243
  350 |      11.1248
  400 |      10.8333
  450 |      10.5495
  500 |      10.2734
  550 |      10.0046
  600 |       9.7430
  650 |       9.4884
  700 |       9.2406
  750 |       8.9995
  800 |       8.7648
  850 |       8.5363
  900 |       8.3140
  950 |       8.0976
 1000 |       7.8870


### 다변량 회귀분석 (행렬 기반)

In [10]:
# 행렬 기반으로 데이터 삽입
# 데이터 정리상태부터 다름

data = np.array([
    # X1,   X2,    X3,   y
    [ 73.,  80.,  75., 152. ],
    [ 93.,  88.,  93., 185. ],
    [ 89.,  91.,  90., 180. ],
    [ 96.,  98., 100., 196. ],
    [ 73.,  66.,  70., 142. ]
], dtype=np.float32)

In [12]:
# 데이터 슬라이싱
X = data[:, :-1]
Y = data[:, [-1]]

In [13]:
# 랜덤하게 W값 설정
W = tf.Variable(tf.random.normal((3,1)))
b = tf.Variable(tf.random.normal((1,)))

In [14]:
# learning_rate 설정
learning_rate = 0.000001

In [16]:
# hypothesis, prediction 함수 구성

def predict(X):
    return tf.matmul(X, W) + b # 이게 지금 hypothesis

print("epoch | cost")

n_epochs = 2000
for i in range(n_epochs + 1):
    with tf.GradientTape() as tape:
        cost = tf.reduce_mean(tf.square(predict(X) - Y)) # cost 값 구현
        
    # 기울기 구하기
    W_grad, b_grad = tape.gradient(cost, [W, b])
    
    # 새롭게 구한 기울기로 각 변수 값 재설정
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 50 == 0:
        print("{:5} | {:10.4f}".format(i, cost.numpy()))

epoch | cost
    0 | 71088.7344
   50 |   796.1061
  100 |    16.1204
  150 |     7.4498
  200 |     7.3378
  250 |     7.3208
  300 |     7.3049
  350 |     7.2890
  400 |     7.2732
  450 |     7.2575
  500 |     7.2418
  550 |     7.2261
  600 |     7.2105
  650 |     7.1949
  700 |     7.1793
  750 |     7.1638
  800 |     7.1484
  850 |     7.1329
  900 |     7.1176
  950 |     7.1022
 1000 |     7.0869
 1050 |     7.0717
 1100 |     7.0564
 1150 |     7.0412
 1200 |     7.0261
 1250 |     7.0110
 1300 |     6.9959
 1350 |     6.9809
 1400 |     6.9659
 1450 |     6.9510
 1500 |     6.9360
 1550 |     6.9212
 1600 |     6.9064
 1650 |     6.8916
 1700 |     6.8769
 1750 |     6.8622
 1800 |     6.8475
 1850 |     6.8328
 1900 |     6.8182
 1950 |     6.8037
 2000 |     6.7892


In [18]:
# W1, W2, W3 각 weight값 구하기
W.numpy()

array([[0.05773982],
       [0.6998049 ],
       [1.2404082 ]], dtype=float32)

In [20]:
# bias 값 구하기
b.numpy()

array([0.5436951], dtype=float32)

In [21]:
# 변수와 bias를 더한 Y값은???
tf.matmul(X, W) + b

<tf.Tensor: id=145259, shape=(5, 1), dtype=float32, numpy=
array([[153.77371],
       [182.85431],
       [181.00153],
       [198.70842],
       [137.77441]], dtype=float32)>

### Predict

In [22]:
Y # labels, 실제값이다.

array([[152.],
       [185.],
       [180.],
       [196.],
       [142.]], dtype=float32)

In [23]:
# prediction, 예측값이다.
predict(X).numpy()

array([[153.77371],
       [182.85431],
       [181.00153],
       [198.70842],
       [137.77441]], dtype=float32)

In [24]:
# 모델을 통해서 새로운 데이터를 넣었을 때 예측을 해본다

predict([[89., 95., 92.], [84., 92., 85.]]).numpy()

array([[186.28157],
       [175.21059]], dtype=float32)